In [1]:
%matplotlib inline
import xarray as xr
import numpy as np 
import glob 
import os 
#import pyarrow as pa 
#from tdigest import TDigest
from crick import TDigest as TDigest_cr 
import matplotlib
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
from multiprocessing import Process
import tqdm

os.chdir('/home/b/b382291/git/AQUA') # CHANGE TO CORRECT PATH OR REMOVE IF USING OTHER DATA 
#imported_module = importlib.import_module("aqua")
from aqua import Reader
from aqua.reader import catalogue

!hostname

l40060.lvt.dkrz.de


# use ICON data

In [2]:

#reader = Reader(model="IFS", exp="tco2559-ng5-cycle3", source="2D_1h_native")
#reader = Reader(model="FESOM", exp="tco3999-ng5", source="interpolated_global_UV")
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09")
reader.reset_stream()
data_gen = reader.retrieve(streaming_generator=True, stream_step=1, stream_unit = "hours")
count = 0 

for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")
    data = data.sp
    break 

start_date: 2020-01-20T00:00:00.000000000 stop_date: 2020-01-20T00:30:00.000000000


In [3]:
data = data.compute() 
ds = data.values[:, 0:10000000] # extract the numpy array 
print(type(ds))
weight, array_length= np.shape(ds)
ds

<class 'numpy.ndarray'>


array([[100823.94 , 101063.91 , 100732.24 , ..., 101509.87 , 101515.016,
        101515.01 ],
       [100824.8  , 101064.89 , 100738.81 , ..., 101515.96 , 101517.055,
        101516.97 ]], dtype=float32)

In [4]:
multiprocessing.cpu_count()

256

initalising digests takes around 17 s 
doing the udpate takes around 1 m 15 s for full array 

for 10 million, init around 10 s, update 35 s (whole thing 42s)

In [31]:
digest_list = []

for j in tqdm.tqdm(range(array_length)):
   # initalising digests 
   digest_list.append(TDigest_cr(compression=1))
   
for j in tqdm.tqdm(range(array_length)):
   # initalising digests 
   digest_list[j].update(ds[:,j])
   

100%|██████████| 10000000/10000000 [00:34<00:00, 294080.14it/s]


In [7]:
digest_list

[TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<compression=20.0, size=2.0>,
 TDigest<com

Using processes 

In [8]:

# Define a function to update the digest for a range of indices

def update_digest(job_length, ds, digest_list):

    [digest_list[j].update(ds[:, j]) for j in range(job_length)]
    print(np.shape(ds))
    print('job_length', job_length)

if __name__ == "__main__":
    
    digest_list = []
    for j in tqdm.tqdm(range(array_length)):
       # initalising digests 
       digest_list.append(TDigest_cr(compression=1))
    
    # Split the work among processes
    num_processes = 22  # this makes it explict 
    chunk_size = array_length // num_processes
    # now adding to the digests   
    processes = []
    pipes = []
      
    for i in range(num_processes):
        #conn1, conn2 = Pipe()
        #pipes.append(conn1)  # Store the connection objects for later use
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        job_length = end - start
        job_args = (job_length, ds[:,start:end], digest_list[start:end])
        process = Process(target=update_digest, args=job_args)
        processes.append(process)

    for process in processes:
        process.start()

    for process in processes:
        process.join()

    # new_results = [y for x in results for y in x]

    # Now 'results' contains the updated 'digest_list' from all processes.

100%|██████████| 10000000/10000000 [00:05<00:00, 1951684.47it/s]


(2, 454545)(2, 454545)(2, 454545)
job_length 

454545
job_lengthjob_length  454545454545

(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)(2, 454545)

job_lengthjob_length  454545454545

(2, 454545)(2, 454545)
job_length
 454545job_length
 454545
(2, 454545)(2, 454545)
(2, 454545)job_length 
454545

job_lengthjob_length  454545454545

(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454545)
job_length 454545
(2, 454555)
job_length 454555


In [9]:
# share numpy array via pipe
from multiprocessing import Process
from multiprocessing import Pipe
from numpy import ones
 
# task executed in a child process
def task(start, end, data, ds, pipe):

    print(data[:5])

    for i in range(start,end):
        #data[i] = ds[1,j]
        data[i].update(ds[:, i])

    # check some data in the array
    print(data[start])
    # send the array via a pipe
    pipe.send(data)
 
# protect the entry point
if __name__ == '__main__':
    
# define the size of the numpy array
    num_processes = 22
    data = []
    for j in tqdm.tqdm(range(array_length)):
        # initalising digests 
        data.append(TDigest_cr(compression=1))
        
    pipes = []
    processes = []
    chunk_size = array_length // num_processes
    
    for i in range(num_processes):
        # create the shared pipe 
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        conn1, conn2 = Pipe()
        pipes.append(conn1)
        process = multiprocessing.Process(target=task, args=(start, end, data, ds, conn2,))
        processes.append(process)   
         
    # start the child process
    for process in processes: 
        process.start()
        
    # Wait for processes tofc finish and collect the results
    data_chunks = []
    
    for process, conn2 in zip(processes, pipes):
        process.join()
        
        try:
            data_chunk = conn2.recv()
            data_chunks.extend(data_chunk)    # read the data from the pipe

        except EOFError as err:
            print('Got here')
        

    # check some data in the array
    print(data_chunks[:5])

100%|██████████| 10000000/10000000 [00:06<00:00, 1532500.67it/s]


[TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>][TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>]

[TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>][TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>]

[TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0.0>]
[TDigest<compression=20.0, size=0.0>, TDigest<compression=20.0, size=0

KeyboardInterrupt: 

Process Process-85:
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2074738/1325765207.py", line 18, in task
    pipe.send(data)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 405, in _send_bytes
    self._send(buf)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
KeyboardInterrupt
Process Process-83:
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", lin

using the one that worked but with a shared memory object

In [13]:
from multiprocessing import Process
from multiprocessing.sharedctypes import Array
from numpy import ones
import ctypes 

# task executed in a child process
def task(array, ds):
    # check some data in the array
    print(array[:10], len(array))
    # change data in the array

    for i in range(len(array)):
        array[i] = ds[1,j]
        #array[i].update(ds[:, i])
        
    # confirm the data was changed
    print(array[300:310], len(array))
    print('changed')
 
# protect the entry point
if __name__ == '__main__':
    
    # define the size of the numpy array
    n = 10000
    # create the numpy array
    data = ones((n,))
    #print(data[:10], data.shape)
    
    data = []
    for j in tqdm.tqdm(range(array_length)):
        # initalising digests 
        data.append(TDigest_cr(compression=1))
        
    #array = RawArray(ctypes.py_object, digest_list)
    # get ctype for our array
    # create ctype array initialized from our array
    array = Array(ctypes.c_wchar_p, data, lock=False)
    # confirm contents of the shared array
    print(array[:10], len(array))
    # create a child process
    child = Process(target=task, args=(array,ds,))
    # start the child process
    child.start()
    # wait for the child process to complete
    child.join()
    # check some data in the shared array
    print(array[:10], len(array))

100%|██████████| 100000/100000 [00:00<00:00, 1087489.08it/s]

TypeError: unicode string or integer address expected instead of crick.tdigest.TDigest instance

In [6]:
array[300]

TDigest<compression=20.0, size=0.0>

In [5]:
from multiprocessing import Queue 

# Define a function to update the digest for a range of indices

def update_digest(queue):

    while True:
        job_length, ds, digest_list = queue.get()
            # check for final item
        if digest_list is None:
            # report message
            print('Task done.', flush=True)
            # push signal back into queue
            queue.put(data)
            # exit the task
            break
        
        [digest_list[j].update(ds[:, j]) for j in range(job_length)]

if __name__ == "__main__":
    
    queue = Queue() 

    digest_list = []
    for j in tqdm.tqdm(range(array_length)):
       # initalising digests 
       digest_list.append(TDigest_cr(compression=1))
    
    # Split the work among processes
    num_processes = 22  # this makes it explict 
    chunk_size = array_length // num_processes
    # now adding to the digests   
    processes = []
    queues = []

    for i in range(num_processes):
        processes = [Process(target=update_digest, args=(queue,)) for _ in range(num_processes)]
        #processes.append(process)

    for process in processes:
        process.start()

    for _ in range(num_processes):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        job_length = end - start
        job_args = (job_length, ds[:,start:end], digest_list[start:end])

        queue.put(job_args)
    queue.put(None)
    
    results = []
    for j in range(len(processes)):
        results.append(queue.get())

    for process in processes:
        process.join()
    print('done join')
    
    #results = []
    #for j in range(len(processes)):
    #    results.append(queue.get())

    print(results)
    # new_results = [y for x in results for y in x]

    # Now 'results' contains the updated 'digest_list' from all processes.

100%|██████████| 10000000/10000000 [00:08<00:00, 1164840.18it/s]
Process Process-480:
Process Process-483:
Process Process-484:
Process Process-482:
Process Process-481:
Process Process-467:
Process Process-463:
Process Process-465:
Process Process-464:
Process Process-466:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.1

KeyboardInterrupt: 

In [11]:
from multiprocessing import Queue 

# Define a function to update the digest for a range of indices

def update_digest(job_length, ds, digest_list, queue):

    [digest_list[j].update(ds[:, j]) for j in range(job_length)]
    #print(np.shape(ds))
    #print('job_length', job_length)
    queue.put(digest_list)

if __name__ == "__main__":
    
    digest_list = []
    for j in tqdm.tqdm(range(array_length)):
       # initalising digests 
       digest_list.append(TDigest_cr(compression=1))
    
    # Split the work among processes
    num_processes = 22  # this makes it explict 
    chunk_size = array_length // num_processes
    # now adding to the digests   
    processes = []
    queues = []

    queue = Queue() 

    for i in range(num_processes):
        #conn1, conn2 = Pipe()
        #queues.append(queue)  # Store the connection objects for later use
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        job_length = end - start
        job_args = (job_length, ds[:,start:end], digest_list[start:end], queue)
        process = Process(target=update_digest, args=job_args)
        processes.append(process)

    for process in processes:
        process.start()

    #for queue in queues: 
        
         
    results = []
    for j in range(len(processes)):
        results.append(queue.get())

    for process in processes:
        process.join()
    print('done join')
    
    #results = []
    #for j in range(len(processes)):
    #    results.append(queue.get())

    print(results)
    # new_results = [y for x in results for y in x]

    # Now 'results' contains the updated 'digest_list' from all processes.

100%|██████████| 5000000/5000000 [00:02<00:00, 1927696.14it/s]


KeyboardInterrupt: 

Now doing the same but using Pipes to communicate the output

In [ ]:
from multiprocessing import Pipe 

# Define a function to update the digest for a range of indices
  
def update_digest(job_length, ds_short, short_list, connection):

    [short_list[j].update(ds_short[:, j]) for j in range(job_length)]
    print(short_list[job_length -1])
    print('job_length', job_length)

    connection.send(short_list)
    #connection.close()
    
if __name__ == "__main__":
        
    digest_list = []
    for j in tqdm.tqdm(range(array_length)):
       # initalising digests 
       digest_list.append(TDigest_cr(compression=1))
    
    # Split the work among processes
    num_processes = 22  # this makes it explict 
    chunk_size = array_length // num_processes
    processes = []
    pipes = []
        
    for i in range(num_processes):
        conn1, conn2 = Pipe()
        pipes.append(conn1)
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        job_length = end - start
        job_args = (job_length, ds[:, start:end], digest_list[start:end], conn2)
        process =  Process(target=update_digest, args=job_args)
        processes.append(process)

    for process in processes:
        process.start()
    
    #[conn2.close() for conn2 in pipes_child]

    for conn1 in pipes:
        #process.join()
        
        try:
            data_chunk = conn1.recv()
            data_chunks.extend(data_chunk)    # read the data from the pipe

        except EOFError as err:
            print('Got here')
            
    for process in processes:
        process.join()
        
    
   # new_results = [y for x in results for y in x]

100%|██████████| 10000000/10000000 [00:04<00:00, 2030334.11it/s]


TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, size=2.0>
job_length 454545
TDigest<compression=20.0, si

Process Process-110:
Process Process-92:
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/tmp/ipykernel_2074738/1944979856.py", line 11, in update_digest
    connection.send(digest_list)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 405, in _send_bytes
    self._send(buf)


  File "/tmp/ipykernel_2074738/1944979856.py", line 11, in update_digest
    connection.send(digest_list)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
Process Process-97:
KeyboardInterrupt
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 405, in _send_bytes
    self._send(buf)
Process Process-103:
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10

In [5]:
from multiprocessing import Queue

# Define a function to update the digest for a range of indices

def init_digest(queue,):
    
    print('building Running', flush=True)
    
    # Split the work among processes
    num_processes = 22  # this makes it explict 
    chunk_size = array_length // num_processes
    
    for i in range(num_processes):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
        queue.put([digest_list[j].update(ds[:, j]) for j in range(start, end)])
    
    queue.put(None)
    print('buildering ready,', Flush = True)

def update_digest(queue,):
    results_new = []
    while True:
        results = queue.get()
        if results is None:
            break
        results_new.append(results)
            
if __name__ == "__main__":

    # with multiprocessing there is a way to do it implicity, more control with the explict 
    # list of dictionaries for each grid cell, preserves order
    digest_list = []

    for j in tqdm.tqdm(range(array_length)):
        # initalising digests 
        digest_list.append(TDigest_cr(compression=1))
    
    queue = Queue()
    process_build = Process(target=init_digest, args=(queue,))
    process_update = Process(target=update_digest, args=(queue,))

    process_build.start()
    process_update.start()
    process_update.join()
    process_build.join()

new_results = [y for x in queue.get() for y in x]

  1%|          | 112387/20971520 [00:00<00:18, 1123820.16it/s]

100%|██████████| 20971520/20971520 [00:17<00:00, 1206046.23it/s]


building Running


Process Process-1:
Traceback (most recent call last):
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/b/b382291/.conda/envs/aqua/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_116633/140856022.py", line 19, in init_digest
    print('buildering ready,', Flush = True)
TypeError: 'Flush' is an invalid keyword argument for print()


KeyboardInterrupt: 

takes 1 min 31 sec sometimes 2 mins (compute node?)

In [7]:
num_processes = 22 # multiprocessing.cpu_count()

# Function to update the digest in parallel
def update_digest(j):
    digest = TDigest_cr(compression=1)
    digest.update(ds[:, j])
    return digest

# Create a pool of worker processes
with multiprocessing.Pool(processes=num_processes) as pool:
    # Use tqdm to monitor progress
    results = list(pool.starmap(update_digest, range(array_length)), total=array_length)

# The results list now contains the updated digests
digest_list = results

TypeError: 'int' object is not iterable

In [8]:
digest_list[7353].centroids()

array([(101683.0546875, 1.), (101685.9140625, 1.)],
      dtype=[('mean', '<f8'), ('weight', '<f8')])

In [9]:
new_results[7353].centroids()

NameError: name 'new_results' is not defined

In [7]:
# Define a function to calculate pu for a given (i, j) position
from multiprocessing import Pool 

#global calculate_power_output

def update_digests(j, digest_list, ds):
    
    digest_list[j].update(ds[:, j])

    return digest_list[j]

digest_list = []

for j in tqdm.tqdm(range(array_length)):
    # initalising digests 
    digest_list.append(TDigest_cr(compression=1))
        
#jobargs = [(j, digest_list, ds) for j in range(array_length)]

# Create a pool with the number of desired processes
num_processes = 22 # multiprocessing.cpu_count()  # Adjust this value based on your system capabilities
#num_processes = multiprocessing.cpu_count()

# Split the work into chunks for parallel processing
chunk_size = array_length // num_processes
chunks = [(j, digest_list, ds) for j in range(array_length)]

with Pool(num_processes) as pool:
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < num_processes - 1 else array_length
    job_length = end - start
    jobargs = [(j, digest_list[start:end], ds[start:end,:]) for j in range(job_length)]
    #job_args = (job_length, ds[start:end,:], digest_list[start:end])
    res = list(pool.starmap(update_digests, jobargs))

pool.close()
pool.join()


100%|██████████| 5000000/5000000 [00:02<00:00, 2066843.80it/s]


: 

: 

In [1]:
import MPI

ModuleNotFoundError: No module named 'MPI'

In [11]:
num_processes = 10 #multiprocessing.cpu_count()

# Function to update the digest in parallel
def update_digest(j, digest_list, ds):
    #digest = TDigest_cr(compression=1)
    digest_list[j].update(ds[:, j])
    return digest_list

job_args = [(i, j, digest_list, ds) for i in range(array_length)]

# Create a pool of worker processes
with multiprocessing.Pool(processes=num_processes) as pool:
    # Use tqdm to monitor progress
    results = list(pool.starmap(update_digest, job_args))


pool.close()
pool.join()
    
# The results list now contains the updated digests
digest_list = results

: 

: 